In [17]:
# Question I'm trying to answer: does the 02:22:55 UT AARTFAAC observation at apparent Ra:257 Dec:+48 match
#                                the Burlage observation at 02:22:55 at apparent Ra:325, Dec:+49

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from astropy.units import Quantity

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation, AltAz, GCRS, ITRS, Angle, SkyCoord, FK5

In [3]:
def radec_to_radec(radec1: SkyCoord, obs1: EarthLocation, obs2: EarthLocation, time: Time, height: Quantity = 100*u.km):
    """Convert apparent Ra/Dec from one site to apparent Ra/Dec from another site"""
    aa1_far = radec1.transform_to(AltAz(location=obs1, obstime=time))
    aa1_near = AltAz(location=obs1, obstime=time, alt=aa1_far.alt, az=aa1_far.az, distance=height/np.sin(aa1_far.alt))
    aa2_near = aa1_near.transform_to(AltAz(location=obs2, obstime=time))
    aa2_far = AltAz(location=obs2, obstime=time, alt=aa2_near.alt, az=aa2_near.az)
    return aa2_far.transform_to(FK5)

In [4]:
t = Time("2020-08-13T02:22:55", format="isot", scale="utc")

In [5]:
height = 100 * u.km

In [6]:
loc_lofar = EarthLocation(lat = 52.9153 * u.deg,
                          lon = 6.8698 * u.deg,
                          height = 20 * u.m)

loc_dwingeloo = EarthLocation(lat = 52.8344 * u.deg,
                              lon = 6.3785 * u.deg,
                              height = 10 * u.m)

loc_burlage = EarthLocation(lat = 53.047010 * u.deg,
                            lon = 7.552926 * u.deg,
                            height = 10 * u.m)

In [7]:
radec_to_radec(FK5(ra=257*u.deg, dec=48*u.deg), loc_lofar, loc_burlage, t)

<FK5 Coordinate (equinox=J2000.000): (ra, dec) in deg
    (264.8387406, 38.98317898)>

In [ ]:
# Conclusion: the AARTFAAC detection is not the same as the one in Burlage

In [9]:
# Check: does radec_to_radec transform back to the input (errors due to flat earth assumption)
radec_to_radec(radec_to_radec(FK5(ra=257*u.deg, dec=48*u.deg), loc_lofar, loc_burlage, t), loc_burlage, loc_lofar, t)

<FK5 Coordinate (equinox=J2000.000): (ra, dec) in deg
    (257.08523051, 47.91843682)>

In [10]:
radec_to_radec(FK5(ra=325*u.deg, dec=48*u.deg), loc_burlage, loc_dwingeloo, t)

<FK5 Coordinate (equinox=J2000.000): (ra, dec) in deg
    (44.05214719, 62.68988115)>

In [16]:
radec_to_radec(FK5(ra=325*u.deg, dec=48*u.deg), loc_burlage, loc_dwingeloo, t, height=np.linspace(90, 120, 10) * u.km)

<FK5 Coordinate (equinox=J2000.000): (ra, dec) in deg
    [(52.58875209, 60.98414671), (49.71224829, 61.64023942),
     (46.86222056, 62.20671475), (44.05214719, 62.68988115),
     (41.29436953, 63.09617628), (38.59981307, 63.43204611),
     (35.97779824, 63.70383789), (33.43594345, 63.91770927),
     (30.98015377, 64.07955473), (28.61468246, 64.19494954)]>